In [3]:
import numpy as np
import random
from matplotlib import pyplot as plt
from collections import Counter
import tensorflow as tf


In [8]:
# Load the dataset (MNIST data)
data = np.loadtxt("zipcombo.dat.txt")

In [9]:
# define binary kernel svm

def kernel(x, x_prime, d):
    x_prime = (x_prime).reshape(len(x_prime),1)
    zero_mat = np.matmul(x, x_prime)**d
    return zero_mat

def prediction(x, x_prime, d, alpha, y, b):
    cal_kernel = kernel(x, x_prime, d)
    cal_kernel = cal_kernel.reshape(len(cal_kernel),1)
    alpha = alpha.reshape(len(alpha),1)
    y = y.reshape(len(y), 1)
    
    prediction = np.sign(np.matmul((alpha * y).T, cal_kernel) + b)
    return prediction[0][0]

def train_svm(X, y, epochs, d, max_passes, C=1):
    n = len(X[:,0])
    alpha = np.zeros((n))
    b = 0
    
    passes = 0
    while (passes < max_passes):
        num_changed_alphas = 0
        prev_alpha = alpha.copy()
        for i in range(n):
            x_i,  y_i= X[i,:],  y[i]
            E_i = prediction(X, x_i, d, alpha, y, b) - y_i
            if ((y_i * E_i < -0.005) and (alpha[i] < C)) or ((y_i * E_i > 0.005) and (alpha[i] > 0)):
                j = random.randint(0,n-1)
                while i == j:
                    j=random.randint(1,n-1)
                x_j, y_j =  X[j,:], y[j]
                E_j = prediction(X, x_j, d, alpha, y, b) - y_j

                old_alpha_i, old_alpha_j = alpha[i].copy(), alpha[j].copy()

                if y_i == y_j:
                    L , H = max(0, old_alpha_i + old_alpha_j - C), min(C, old_alpha_i + old_alpha_j)
                else:
                    L , H = max(0, old_alpha_i - old_alpha_j), min(C, C + old_alpha_i - old_alpha_j)
                if L == H:
                    continue

                eta =  2 * np.dot(x_i, x_j)**d - np.dot(x_i, x_i)**d - np.dot(x_j, x_j)**d
                if eta >= 0:
                    continue        


                alpha[j] -= float(y_j * (E_i - E_j))/eta
                if alpha[j] > H:
                    alpha[j] == H
                elif L <= alpha[j] <= H:
                    alpha[j] = alpha[j]
                elif alpha[j] < L:
                    alpha[j] = L

                alpha[i] += y_i*y_j * (old_alpha_j - alpha[j])

                b1 = b - E_i - y_i * (alpha[i] - old_alpha_i) * np.dot(x_i, x_i)**d - y_j * (alpha[j] - old_alpha_j) * np.dot(x_i, x_j)**d
                b2 = b - E_j - y_i * (alpha[i] - old_alpha_i) * np.dot(x_i, x_j)**d - y_j * (alpha[j] - old_alpha_j) * np.dot(x_j, x_j)**d

                if 0 < alpha[i] and C > alpha[i]:
                    b = b1
                elif 0 < alpha[j] and C > alpha[j]:
                    b = b2
                else:
                    b = (b1 + b2) / 2.0
                    
                num_changed_alphas += 1
        if num_changed_alphas == 0:
            passes += 1
        else:
            passes = 0
    return alpha, b

In [10]:
def Most_Common(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]

def onevsone(data, d, data_test, max_passes = 100):
    data_copy = data.copy()
    data_test_copy = data_test.copy()
    data_list = []
    data_test_list = []
    combinations = []
    for index, item in enumerate(np.arange(10), 0):
        temp = (np.arange(9)[index:])
        for i in temp:
            combinations.append([index, i+1])
    
    for i in range(45):
        first = combinations[i][0]
        second = combinations[i][1]


        data_temp1 = (data_copy[data_copy[:,0] == first])
        data_temp2 = (data_copy[data_copy[:,0] == second])
        data_temp_2 = np.vstack((data_temp1, data_temp2))        
        np.random.shuffle(data_temp_2)
        
        data_temp_2[:,0][data_temp_2[:,0] == first] = -1
        data_temp_2[:,0][data_temp_2[:,0] == second] = 1
        
        data_list.append(data_temp_2)

        
    vote_list = np.zeros((45, len(data_test[:,0])))

    for i in range(45):
        print(combinations[i])
        x, y = data_list[i][:,1:], data_list[i][:,0]
        x_test, y_test = data_test[:, 1:], data_test[:,0]
        
        output = train_svm(x, y, 10, d, 100)
        
        test = []
        for j in range(len(y_test)):
            test.append(prediction(x, x_test[j], d, output[0], y, output[1]))
        test = np.array(test)
        idx_1 = (test == -1.0)
        idx_2 = (test == 1.0)
        test[idx_1] = combinations[i][0]
        test[idx_2] = combinations[i][1]
        vote_list[i,:] = test

    return vote_list

In [33]:
output = (onevsone(data[0:2000], 2, data[2000:2500]))

[0, 1]
[0, 2]
[0, 3]
[0, 4]
[0, 5]
[0, 6]
[0, 7]
[0, 8]
[0, 9]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[1, 7]
[1, 8]
[1, 9]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
[2, 8]
[2, 9]
[3, 4]
[3, 5]
[3, 6]
[3, 7]
[3, 8]
[3, 9]
[4, 5]
[4, 6]
[4, 7]
[4, 8]
[4, 9]
[5, 6]
[5, 7]
[5, 8]
[5, 9]
[6, 7]
[6, 8]
[6, 9]
[7, 8]
[7, 9]
[8, 9]


In [34]:
test = []
for i in range(len(output[0,:])):
    test.append(Most_Common(output[:,i]))

In [35]:
np.sum(np.array(test) !=  np.array(data[2000:2500, 0]))/len(data[2000:2500, 0])

0.078

In [37]:
print("Test Accuracy: ", 1-0.078)

Test Accuracy:  0.922
